<a href="https://colab.research.google.com/github/KieranPereira/SkinBurnClassification/blob/main/Studentship_Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing Dependencies
import torch
import os
import shutil
import zipfile
import time
import yaml
%pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Cloning yolov5 repository and installing additional dependencies
if os.path.exists("yolov5")==False:

    print("preparing to clone yolov5 repository")
    time.sleep(1)

    !git clone https://github.com/ultralytics/yolov5
    os.chdir("yolov5")
    #Installing Dependencies

    print("preparing to install dependencies")
    time.sleep(1)

    %pip install -r requirements.txt
    os.chdir("..")


In [ ]:
#Creating dataset paths
if os.path.exists("BurnsDataset")==False:
    
    print("preparing to create dataset paths")
    
  
    os.makedirs("BurnsDataset")
    os.chdir("BurnsDataset")
    os.makedirs("test/images")
    os.makedirs("test/labels")
    os.makedirs("train/images")
    os.makedirs("train/labels")
    os.makedirs("val/images")
    os.makedirs("val/labels")
    os.chdir("..")

preparing to create dataset paths


In [ ]:
while os.getcwd()!="/content":
  os.chdir("..")
print(os.getcwd())

/content


In [ ]:
#Downloading Dataset from kaggle.com. Requires Kaggle API token to be downloaded from website (you must create an account for this)
if os.path.exists("..\.kaggle\kaggle.json")==False:
    
    print("preparing to download dataset")
    time.sleep(2)
    !mkdir ~/.kaggle
    from google.colab import files
    files.upload()
    shutil.move("kaggle.json","../root/.kaggle/kaggle.json")
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d shubhambaid/skin-burn-dataset
    with zipfile.ZipFile("skin-burn-dataset.zip","r") as zip_ref:
        zip_ref.extractall("BurnsDataset")
    os.remove("skin-burn-dataset.zip")

preparing to download dataset


Saving kaggle.json to kaggle.json
 30% 5.00M/16.6M [00:00<00:00, 37.0MB/s]
100% 16.6M/16.6M [00:00<00:00, 84.8MB/s]


In [ ]:
    #Dataset formatting
    os.chdir("BurnsDataset")
    print("preparing to format data")
    time.sleep(5)
    #Removing unlabelled images from dataset
    #Counting number of images in new dataset
    number_img = 0
    text_img =0
    for i in range (0,1441):
        string1= str(i)
        if os.path.exists("img"+string1+".jpg")==True:
            number_img=number_img+1
    print("There are {} images in your dataset".format(string1))
    test_ratio=float(input("Please enter the ratio for your test data (note test, val, train data must sum to 10)"))
    val_ratio=float(input("Please enter the ratio for your val data (note test, val, train data must sum to 10)"))
    train_ratio=float(input("Please enter the ratio for your train data (note test, val, train data must sum to 10)"))
    while test_ratio+val_ratio+train_ratio !=10:
        print(" test, val, train data must sum to 10")
        test_ratio=float(input("Please enter the ratio for your test data (note test, val, train data must sum to 10)"))
        val_ratio=float(input("Please enter the ratio for your val data (note test, val, train data must sum to 10)"))
        train_ratio=float(input("Please enter the ratio for your train data (note test, val, train data must sum to 10)"))
    print("splitting into " +str(test_ratio)+":"+str(val_ratio)+":" +str(train_ratio)+" for test:val:train")
    time.sleep(2)
    test_num= number_img//(float(1/(test_ratio/10)))
    val_num= number_img//(float(1/(val_ratio/10)))
    train_num= number_img//(float(1/(train_ratio/10)))
    # Creating a function that creates a list of files in a directory
    while os.getcwd()!="/content":
      os.chdir("..")
    print(os.getcwd())
# create a list of files in burns dataset
# names in the given directory 
    allFiles=[]
    for entry in os.listdir("BurnsDataset"):
        if ".jpg" in entry:
            allFiles.append(entry)
        if ".jpeg" in entry:
            allFiles.append(entry)
    with open("allfiles.txt","w") as w:
        w.write(str(allFiles))
    w.close
    te_num=0
    va_num=0
    tr_num=0
    os.chdir("BurnsDataset")
    print("appending files to test folder: \n {}".format(os.getcwd()))
    time.sleep(5)
    for i in range(0,len(allFiles)):
        if i in range(0,int(test_num)):
            te_num=te_num+1
            entry_file= allFiles[i]
            if ".jpg" in entry_file:
                shutil.move(entry_file, "test")
                shutil.move(str(entry_file).replace(".jpg",".txt"), "test")
            elif ".jpeg" in entry_file:
                shutil.move(entry_file, "test")
                shutil.move(str(entry_file).replace(".jpeg",".txt"),"test")
        os.chdir("test")
        for filename in os.listdir():
            if ".jpg" in filename:
                shutil.move(filename,"images")
            elif ".txt" in filename:
                shutil.move(filename,"labels")
            elif ".jpeg" in filename:
                shutil.move(filename,"images")
        os.chdir("..")
    print("te_num={}".format(te_num))
    print("appending files to validation folder: \n {}".format(os.getcwd()))
    time.sleep(5)
    for i in range(0,len(allFiles)):
        if i in range(int(test_num),int(val_num*2)):
            va_num=va_num+1
            entry_file= allFiles[i]
            if ".jpg" in entry_file:
                shutil.move(entry_file, "val")
                shutil.move(str(entry_file).replace(".jpg",".txt"), "val")
            elif ".jpeg" in entry_file:
                shutil.move(entry_file, "val")
                shutil.move(str(entry_file).replace(".jpeg",".txt"), "val")
        os.chdir("val")
        for filename in os.listdir():
            if ".jpg" in filename:
                shutil.move(filename,"images")
            elif ".txt" in filename:
                shutil.move(filename,"labels")
            elif ".jpeg" in filename:
                shutil.move(filename,"images")
        os.chdir("..")
    print("va_num={}".format(va_num))
    print("appending files to train folder: \n {}".format(os.getcwd()))
    time.sleep(5)
    for i in range(0,len(allFiles)):
        if i in range(int(val_num*2),len(allFiles)):
            tr_num=tr_num+1
            entry_file= allFiles[i]
            if ".jpg" in entry_file:
                shutil.move(entry_file, "train")
                shutil.move(str(entry_file).replace(".jpg",".txt"), "train")
            elif ".jpeg" in entry_file:
                shutil.move(entry_file, "train")
                shutil.move(str(entry_file).replace(".jpeg",".txt"), "train")
        os.chdir("train")
        for filename in os.listdir():
            if ".jpg" in filename:
                shutil.move(filename,"images")
            elif ".txt" in filename:
                shutil.move(filename,"labels")
            elif ".jpeg" in filename:
                shutil.move(filename,"images")
        os.chdir("..")
    for file in os.listdir():
        if ".txt" in file:
            os.remove(file)
    print("Dataset Successfully Sorted")

preparing to format data
There are 1440 images in your dataset
Please enter the ratio for your test data (note test, val, train data must sum to 10)0.5
Please enter the ratio for your val data (note test, val, train data must sum to 10)1.5
Please enter the ratio for your train data (note test, val, train data must sum to 10)8
splitting into 0.5:1.5:8.0 for test:val:train
/content
appending files to test folder: 
 /content/BurnsDataset
te_num=61
appending files to validation folder: 
 /content/BurnsDataset
va_num=305
appending files to train folder: 
 /content/BurnsDataset
Dataset Successfully Sorted


In [ ]:
#Creating Custom Yaml file for data paths
while os.getcwd()!="/content":
  os.chdir("..")
print(os.getcwd())
if os.path.exists("yolov5/data/custom.yaml") == False:
    print("Creating custom yaml file:")
    time.sleep(2)
    with open('custom.yaml', 'w+') as file:
        file.write(
"""
path: ../BurnsDataset
train: train/images
test: test/images
val: val/images
nc: 3
names: ["1st Degree","2nd Degree","3rd Degree"]
"""
        )
        file.close()
    shutil.move("custom.yaml","yolov5/data/custom.yaml")

/content
Creating custom yaml file:


In [ ]:
num_classes = 3
##write custom model .yaml
with open('custom_yolov5s.yaml', 'w') as f:
  # parameters
  f.write('nc: ' + str(num_classes) + '\n')
  #f.write('nc: ' + str(len(class_labels)) + '\n')
  f.write('depth_multiple: 0.33'  + '\n') # model depth multiple
  f.write('width_multiple: 0.50'  + '\n')  # layer channel multiple
  f.write('\n')
  f.write('anchors:' + '\n')
  f.write('  - [10,13, 16,30, 33,23] ' + '\n')
  f.write('  - [30,61, 62,45, 59,119]' + '\n')
  f.write('  - [116,90, 156,198, 373,326] ' + '\n')
  f.write('\n')

  f.write('backbone:' + '\n')
  f.write('  [[-1, 1, Focus, [64, 3]],' + '\n') #0-P1/2
  f.write('   [-1, 1, Conv, [128, 3, 2]],' + '\n')# 1-P2/4
  f.write('   [-1, 3, BottleneckCSP, [128]],' + '\n')
  f.write('   [-1, 1, Conv, [256, 3, 2]],' + '\n')# 3-P3/8
  f.write('   [-1, 9, BottleneckCSP, [128]],' + '\n')
  f.write('   [-1, 1, Conv, [512, 3, 2]], ' + '\n')# 5-P4/16
  f.write('   [-1, 9, BottleneckCSP, [256]],' + '\n')
  f.write('   [-1, 1, Conv, [1024, 3, 2]],' + '\n')# 7-P5/32
  f.write('   [-1, 1, SPP, [1024, [5, 9, 13]]],' + '\n')
  f.write('   [-1, 3, BottleneckCSP, [1024,False]],' + '\n') # 9
  f.write('  ]' + '\n')
  f.write('\n')

  f.write('head:'  + '\n')
  f.write('  [[-1, 1, Conv, [512, 1, 1]],'  + '\n')
  f.write('   [-1, 1, nn.Upsample, [None, 2,\'nearest\']],' + '\n')
  f.write('   [[-1, 6], 1, Concat, [1]],' + '\n')#Cat Backbone P4
  f.write('   [-1, 3, BottleneckCSP, [512, False]],' + '\n')# 13
  f.write('\n' )
  f.write('   [-1, 1, Conv, [256, 1, 1]],'  + '\n')
  f.write('   [-1, 1, nn.Upsample, [None, 2, \'nearest\']],' + '\n')
  f.write('   [[-1, 4], 1, Concat, [1]],' + '\n')# cat backbone P3
  f.write('   [-1, 3, BottleneckCSP, [256, False]],' + '\n') # 17 (P3/8-small)
  f.write('\n' )
  f.write('   [-1, 1, Conv, [256, 3, 2]],' + '\n')
  f.write('   [[-1, 14], 1, Concat, [1]],' + '\n')# cat head P4
  f.write('   [-1, 3, BottleneckCSP, [512, False]],' + '\n')# 23 (P5/32-large)
  f.write('\n' )
  f.write('   [-1, 1, Conv, [512, 3, 2]],' + '\n')
  f.write('   [[-1, 10], 1, Concat, [1]],' + '\n')# cat head P5
  f.write('   [-1, 3, BottleneckCSP, [1024, False]],' + '\n')# 23 (P5/32-large)
  f.write('\n' )
  f.write('   [[17, 20, 23], 1, Detect, [nc, anchors]],' + '\n')# Detect(P3, P4, P5)
  f.write('  ]' + '\n')

print('custom model config written!')
shutil.move("custom_yolov5s.yaml", "/content/yolov5/models/custom_yolov5s.yaml")

custom model config written!


In [ ]:
print(os.getcwd())

/content


In [ ]:
#Changing Directory to facilitate training
while os.getcwd()!="/content":
  os.chdir("..")
os.chdir("yolov5")
print(os.getcwd())

/content/yolov5


In [ ]:
#Training custom model
!python train.py --data custom.yaml --cfg custom_yolov5s.yaml --weights '' --batch-size 64 --epochs 600 --img 640 --cache 

In [ ]:
#Hyperparameter optimisation of custom model (must be done after initial training has completed)
!python train.py --data custom.yaml --resume /content/yolov5/runs/train/exp5/weights/last.pt --batch-size 64 --epochs 600 --img 640 --cache --hyp hyp.scratch-high.yaml --evolve

In [ ]:
#Training using pre-trained weights from YOLOV5 repository
!python train.py --data custom.yaml --weights yolov5s.pt --batch-size 32 --epochs 100 --img 640 --cache 